[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

> This notebook runs YouTubeDNN on MovieLens dataset.

# Download dataset and install packages

In [1]:
!git clone https://github.com/Wp-Zhang/HandyRec.git
! pip install faiss-cpu

Cloning into 'HandyRec'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 84 (delta 24), reused 66 (delta 9), pack-reused 0
Unpacking objects: 100% (84/84), done.
     |████████████████████████████████| 8.6 MB 14.4 MB/s 


In [2]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip
! unzip -o ml-1m.zip

--2022-03-10 22:51:22--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  6.83MB/s    in 0.8s    

2022-03-10 22:51:24 (6.83 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


# Import relative packages

In [3]:
import sys
sys.path.append('./HandyRec/')

In [4]:
from handyrec.examples.utils import DataProcessor, sampledsoftmaxloss
from handyrec.models.match import YouTubeDNN
from handyrec.features import DenseFeature, SparseFeature, SparseSeqFeature
import tensorflow as tf
from tensorflow.keras import Model
import numpy as np
import faiss

In [5]:
import warnings
warnings.filterwarnings('ignore')

# Load and prepare data

In [6]:
EMBEDDING_DIM = 128
SEQ_LEN = 40
BATCH_SIZE = 2**12
NEPOCH = 100

In [7]:
dp = DataProcessor('./ml-1m/')

In [8]:
features = ['user_id', 'movie_id', 'gender', 'age', 'occupation', 'zip']
lbd_features = ['gender','age','occupation', 'zip']
data = dp.preprocess_data(lbd_features)
dp.gen_data_set(features, data, seq_max_len=SEQ_LEN, negnum=0)

Encode User Sparse Feats: 100%|██████████| 4/4 [00:00<00:00, 179.30it/s]
Encode Item Sparse Feats: 0it [00:00, ?it/s]
100%|██████████| 4/4 [00:00<00:00,  7.20it/s]
0it [00:00, ?it/s]


In [9]:
user_features = ['user_id','gender','age','occupation', 'zip']
movie_features = ['movie_id']
train, train_label, test, test_label = dp.load_dataset(user_features, movie_features)

Load movie Features: 100%|██████████| 1/1 [00:00<00:00, 294.48it/s]


# Train model and export embeddings

In [10]:
feature_dim = {}
for feat in user_features:
    feature_dim[feat] = data['user'][feat].max()+1
for feat in movie_features:
    feature_dim[feat] = data['item'][feat].max()+1

In [11]:
user_dense_feats = []
user_sparse_feats = ['user_id','gender','age','occupation', 'zip']

In [12]:
user_features = [SparseFeature(x, feature_dim[x], EMBEDDING_DIM) for x in user_sparse_feats] +\
                [DenseFeature(x) for x in user_dense_feats] +\
                [SparseSeqFeature(SparseFeature('movie_id', feature_dim['movie_id'], EMBEDDING_DIM), 'hist_movie_id',SEQ_LEN)]
item_id = SparseFeature('movie_id', feature_dim['movie_id'], EMBEDDING_DIM)

In [13]:
tf.compat.v1.disable_eager_execution()

In [14]:
model = YouTubeDNN(
    user_features, item_id, num_sampled=100, 
    user_dnn_hidden_units=(256,128,EMBEDDING_DIM), dnn_dropout=0.2
)

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
model.compile(optimizer="Adam", loss=sampledsoftmaxloss)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
history = model.fit(train, train_label,
                    batch_size=BATCH_SIZE, 
                    epochs=NEPOCH,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stop])
model.save_weights('youtubednn.h5')

Train on 845828 samples, validate on 93981 samples
Epoch 1/100
845828/845828 [==============================] - 16s 18us/sample - loss: 3.5403 - val_loss: 3.4257
Epoch 2/100
845828/845828 [==============================] - 8s 10us/sample - loss: 2.9143 - val_loss: 2.9377
Epoch 3/100
845828/845828 [==============================] - 8s 10us/sample - loss: 2.6262 - val_loss: 2.4872
Epoch 4/100
845828/845828 [==============================] - 8s 10us/sample - loss: 2.4222 - val_loss: 2.3271
Epoch 5/100
845828/845828 [==============================] - 8s 10us/sample - loss: 2.2784 - val_loss: 2.2241
Epoch 6/100
845828/845828 [==============================] - 8s 10us/sample - loss: 2.1628 - val_loss: 2.1728
Epoch 7/100
845828/845828 [==============================] - 8s 10us/sample - loss: 2.0946 - val_loss: 2.0734
Epoch 8/100
845828/845828 [==============================] - 8s 10us/sample - loss: 1.9985 - val_loss: 2.0243
Epoch 9/100
845828/845828 [==============================] - 8s 10us

In [16]:
model.load_weights('youtubednn.h5')

In [17]:
all_item_model_input = {"movie_id": data['item']['movie_id'].values}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test, batch_size=2 ** 12)
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

(6040, 128)
(3883, 128)


# Use Faiss to predict

In [18]:
index = faiss.IndexFlatIP(EMBEDDING_DIM)
index.add(item_embs)

In [19]:
D, I = index.search(np.ascontiguousarray(user_embs), 10)
s = []
for i, uid in enumerate(test['user_id']):
    try:
        pred = data['item']['movie_id'].values[I[i]].tolist()
        s.append(pred)
    except:
        print(i)
s = np.array(s)

In [20]:
def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k=12):
    return np.mean(
        [apk(a, p, k) for a, p in zip(actual, predicted)]
    )


def rk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = sum([1 for r in actual if r in predicted])/len(actual)

    return score

def recall_at_k(actual, predicted, k=12):
    return np.mean(
        [rk(a, p, k) for a, p in zip(actual, predicted)]
    )

In [21]:
mapk(test_label, s, k=10)

0.026905721118469464

In [22]:
recall_at_k(test_label, s, k=10)

0.07466887417218543